# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [635]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Generate the mathematical model

In [651]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [652]:
#Show the resopnse in a formatted way
Markdown(response.text)

Let's define the variables:

*  $x_1$: acres of land devoted to wheat
*  $x_2$: acres of land devoted to corn
*  $x_3$: acres of land devoted to sugar beets

**Objective function:**

The objective is to minimize the total cost. The total cost is composed of planting costs, purchasing costs (if any), and negative revenue from sales. 

* **Planting costs:** $150x_1 + 230x_2 + 260x_3$
* **Purchasing cost of wheat:** $170(1+40/100)(200-2.5x_1)$ if $2.5x_1<200$,  $0$ otherwise. 
* **Purchasing cost of corn:** $150(1+40/100)(240-3x_2)$ if $3x_2<240$, $0$ otherwise.
* **Revenue from wheat sales:** $-170(2.5x_1-200)$ if $2.5x_1>200$, $0$ otherwise.
* **Revenue from corn sales:** $-150(3x_2-240)$ if $3x_2>240$, $0$ otherwise.
* **Revenue from sugar beets sales within quota:** $-36 \min(20x_3, 6000)$ 
* **Revenue from sugar beets sales exceeding quota:** $-10 (20x_3-6000)$ if $20x_3>6000$, $0$ otherwise. 

Therefore the objective function is:

$ \min\  150x_1 + 230x_2 + 260x_3 + 238(200-2.5x_1) \mathbb{I}_{2.5x_1<200} + 210(240-3x_2) \mathbb{I}_{3x_2<240} - 170(2.5x_1-200) \mathbb{I}_{2.5x_1>200} - 150(3x_2-240) \mathbb{I}_{3x_2>240} - 36 \min(20x_3, 6000)  -10 (20x_3-6000) \mathbb{I}_{20x_3>6000} $

where $\mathbb{I}$ denotes the indicator function that equals $1$ if the subscript condition is satisfied, and equals $0$ otherwise.

**Constraints:**

* **Land constraint:** $x_1+x_2+x_3\leq 500$
* **Non-negativity constraints:** $x_1\geq 0$, $x_2\geq 0$, $x_3\geq 0$

**Mathematical Optimization Model:**

$ \min\  150x_1 + 230x_2 + 260x_3 + 238(200-2.5x_1) \mathbb{I}_{2.5x_1<200} + 210(240-3x_2) \mathbb{I}_{3x_2<240} - 170(2.5x_1-200) \mathbb{I}_{2.5x_1>200} - 150(3x_2-240) \mathbb{I}_{3x_2>240} - 36 \min(20x_3, 6000)  -10 (20x_3-6000) \mathbb{I}_{20x_3>6000} $

Subject to:
$x_1+x_2+x_3\leq 500$
$x_1\geq 0$
$x_2\geq 0$
$x_3\geq 0$


## 3. Generate the pyomo code

In [653]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [654]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def objective_rule(model):
    # Planting costs
    planting_cost = 150*model.x1 + 230*model.x2 + 260*model.x3
    
    # Purchasing cost of wheat
    wheat_purchase_cost = 238*(200 - 2.5*model.x1) if 2.5*model.x1 < 200 else 0
    
    # Purchasing cost of corn
    corn_purchase_cost = 210*(240 - 3*model.x2) if 3*model.x2 < 240 else 0 
    
    # Revenue from wheat sales
    wheat_revenue = -170*(2.5*model.x1 - 200) if 2.5*model.x1 > 200 else 0
    
    # Revenue from corn sales
    corn_revenue = -150*(3*model.x2 - 240) if 3*model.x2 > 240 else 0
    
    # Revenue from sugar beets sales within quota
    sugar_beets_revenue_within_quota = -36 * min(20*model.x3, 6000)
    
    # Revenue from sugar beets sales exceeding quota
    sugar_beets_revenue_exceeding_quota = -10*(20*model.x3 - 6000) if 20*model.x3 > 6000 else 0

    # Total cost
    total_cost = planting_cost + wheat_purchase_cost + corn_purchase_cost + wheat_revenue + corn_revenue + sugar_beets_revenue_within_quota + sugar_beets_revenue_exceeding_quota 
    
    return total_cost

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define the constraints
model.land_constraint = pyo.Constraint(expr=model.x1 + model.x2 + model.x3 <= 500)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal solution:")
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("x3 =", model.x3.value)
print("Total cost =", model.objective.expr()) 
``` 


## 4. Input problem data and try running the generated code

In [655]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def objective_rule(model):
    # Planting costs
    planting_cost = 150*model.x1 + 230*model.x2 + 260*model.x3
    
    # Purchasing cost of wheat
    wheat_purchase_cost = 238*(200 - 2.5*model.x1) if 2.5*model.x1 < 200 else 0
    
    # Purchasing cost of corn
    corn_purchase_cost = 210*(240 - 3*model.x2) if 3*model.x2 < 240 else 0 
    
    # Revenue from wheat sales
    wheat_revenue = -170*(2.5*model.x1 - 200) if 2.5*model.x1 > 200 else 0
    
    # Revenue from corn sales
    corn_revenue = -150*(3*model.x2 - 240) if 3*model.x2 > 240 else 0
    
    # Revenue from sugar beets sales within quota
    sugar_beets_revenue_within_quota = -36 * min(20*model.x3, 6000)
    
    # Revenue from sugar beets sales exceeding quota
    sugar_beets_revenue_exceeding_quota = -10*(20*model.x3 - 6000) if 20*model.x3 > 6000 else 0

    # Total cost
    total_cost = planting_cost + wheat_purchase_cost + corn_purchase_cost + wheat_revenue + corn_revenue + sugar_beets_revenue_within_quota + sugar_beets_revenue_exceeding_quota 
    
    return total_cost

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define the constraints
model.land_constraint = pyo.Constraint(expr=model.x1 + model.x2 + model.x3 <= 500)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal solution:")
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("x3 =", model.x3.value)
print("Total cost =", model.objective.expr()) 

ERROR: Rule failed when generating expression for Objective objective with
index None: PyomoException: Cannot convert non-constant Pyomo expression
(2.5*x1  <  200) to bool. This error is usually caused by using a Var, unit,
or mutable Param in a Boolean context such as an "if" statement, or when
checking container membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'objective' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression (2.5*x1
        <  200) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y

PyomoException: Cannot convert non-constant Pyomo expression (2.5*x1  <  200) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

In [658]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)

# Define indicator variables
model.indicator_wheat_purchase = pyo.Var(domain=pyo.Binary)
model.indicator_corn_purchase = pyo.Var(domain=pyo.Binary)
model.indicator_wheat_revenue = pyo.Var(domain=pyo.Binary)
model.indicator_corn_revenue = pyo.Var(domain=pyo.Binary)
model.indicator_sugar_beets_within_quota = pyo.Var(domain=pyo.Binary)
model.indicator_sugar_beets_exceeding_quota = pyo.Var(domain=pyo.Binary)

# Define auxiliary variables for min function
model.min_sugar_beets_quota = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def objective_rule(model):
    # Planting costs
    planting_cost = 150*model.x1 + 230*model.x2 + 260*model.x3
    
    # Purchasing cost of wheat
    wheat_purchase_cost = 238*(200 - 2.5*model.x1)
    
    # Purchasing cost of corn
    corn_purchase_cost = 210*(240 - 3*model.x2)
    
    # Revenue from wheat sales
    wheat_revenue = -170*(2.5*model.x1 - 200)
    
    # Revenue from corn sales
    corn_revenue = -150*(3*model.x2 - 240)
    
    # Revenue from sugar beets sales within quota
    sugar_beets_revenue_within_quota = -36 * model.min_sugar_beets_quota
    
    # Revenue from sugar beets sales exceeding quota
    sugar_beets_revenue_exceeding_quota = -10*(20*model.x3 - 6000)
    
    # Total cost
    total_cost = planting_cost + wheat_purchase_cost + corn_purchase_cost + wheat_revenue + corn_revenue + sugar_beets_revenue_within_quota + sugar_beets_revenue_exceeding_quota 
    
    # Use indicator variables to control the terms in the objective function
    return total_cost + 10000*(model.indicator_wheat_purchase + model.indicator_corn_purchase + model.indicator_wheat_revenue + model.indicator_corn_revenue + model.indicator_sugar_beets_within_quota + model.indicator_sugar_beets_exceeding_quota)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define the constraints
model.land_constraint = pyo.Constraint(expr=model.x1 + model.x2 + model.x3 <= 500)

# Define the constraint for minimum sugar beets quota
model.min_sugar_beets_constraint = pyo.Constraint(expr=model.min_sugar_beets_quota == min(20*model.x3, 6000))

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal solution:")
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("x3 =", model.x3.value)
print("Total cost =", model.objective.expr())


PyomoException: Cannot convert non-constant Pyomo expression (6000  <  20*x3) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [659]:
print(response.text)

Let's define the variables:

*  $x_1$: acres of land devoted to wheat
*  $x_2$: acres of land devoted to corn
*  $x_3$: acres of land devoted to sugar beets

**Objective function:**

The objective is to minimize the total cost. The total cost is composed of planting costs, purchasing costs (if any), and negative revenue from sales. 

* **Planting costs:** $150x_1 + 230x_2 + 260x_3$
* **Purchasing cost of wheat:** $170(1+40/100)(200-2.5x_1)$ if $2.5x_1<200$,  $0$ otherwise. 
* **Purchasing cost of corn:** $150(1+40/100)(240-3x_2)$ if $3x_2<240$, $0$ otherwise.
* **Revenue from wheat sales:** $-170(2.5x_1-200)$ if $2.5x_1>200$, $0$ otherwise.
* **Revenue from corn sales:** $-150(3x_2-240)$ if $3x_2>240$, $0$ otherwise.
* **Revenue from sugar beets sales within quota:** $-36 \min(20x_3, 6000)$ 
* **Revenue from sugar beets sales exceeding quota:** $-10 (20x_3-6000)$ if $20x_3>6000$, $0$ otherwise. 

Therefore the objective function is:

$ \min\  150x_1 + 230x_2 + 260x_3 + 238(200-2.5x_1) 

In [660]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def objective_rule(model):
    # Planting costs
    planting_cost = 150*model.x1 + 230*model.x2 + 260*model.x3
    
    # Purchasing cost of wheat
    wheat_purchase_cost = 238*(200 - 2.5*model.x1) if 2.5*model.x1 < 200 else 0
    
    # Purchasing cost of corn
    corn_purchase_cost = 210*(240 - 3*model.x2) if 3*model.x2 < 240 else 0 
    
    # Revenue from wheat sales
    wheat_revenue = -170*(2.5*model.x1 - 200) if 2.5*model.x1 > 200 else 0
    
    # Revenue from corn sales
    corn_revenue = -150*(3*model.x2 - 240) if 3*model.x2 > 240 else 0
    
    # Revenue from sugar beets sales within quota
    sugar_beets_revenue_within_quota = -36 * min(20*model.x3, 6000)
    
    # Revenue from sugar beets 